In [1]:
from seeq import spy
import pandas as pd
import numpy as np

In [2]:
# Log into Seeq Server if you're not using Seeq Data Lab:
spy.login(url='http://localhost:34216', credentials_file='../../credentials.key', force=False)

In [3]:
from seeq.spy.assets import Asset

class Area(Asset):
    def initialize(self):
        # These are effectively x,y coordinates proportional to the
        # map of the U.S. in this folder.
        map_positions = {
            # Cooling Tower 1
            'A': ['15%', '4%'],
            'B': ['45%', '60%'],
            'C': ['35%', '35%'],
            'G': ['70%', '60%'],
            'H': ['32%', '80%'],
            'I': ['55%', '40%'],
            'J': ['15%', '34%'],
            'K': ['80%', '6%'],
            
            # Cooling Tower 2
            'D': ['40%', '50%'],
            'E': ['50%', '60%'],
            'F': ['60%', '70%']
        }
        
        self.map_position = map_positions[self['Name']]
    
    @Asset.Attribute()
    def Incidents(self, metadata):
        return {
            'Type': 'Condition',
            'Formula': '$a.validValues().valueSearch(isGreaterThan(120))',
            'Formula Parameters': {
                '$a': metadata[metadata['Name'] == 'Temperature']
            }
        }
    
    @Asset.Attribute()
    def Incident_Count(self, metadata):
        return {
            'Type': 'Metric',
            'Measured Item': self.Incidents(),
            'Measured Item Maximum Duration': '40d',
            'Statistic': 'count',
            'Thresholds': {
                'HiHiHi': 1,
                'HiHi': 0
            }
        }
    
    @Asset.Display()
    def Incident_Display(self, metadata, analysis):
        worksheet = analysis.worksheet('Area ' + self.definition['Name'])
        
        workstep = worksheet.workstep()
        workstep.display_items = [{
            'Item': self.Incident_Count()
        }]
        workstep.view = 'Scorecard'
        workstep.data['state']['stores']['sqTrendMetricStore']['scorecardHeaders'] = {
          "type": "none"
        }
        workstep.data['state']['stores']['sqTrendMetricStore']['scorecardColumns'] = list()
        
        return workstep
    
    @Asset.Display()
    def Detail(self, metadata, analysis):
        worksheet = analysis.worksheet('Area ' + self.definition['Name'])
        
        workstep = worksheet.workstep()
        workstep.display_items = metadata[metadata['Name'] == 'Temperature']
        workstep.view = 'Trend'
        
        return workstep

class Cooling_Tower(Asset):
    @Asset.Component()
    def Areas(self, metadata):
        return self.build_components(Area, metadata, column_name='Area')
    
    @Asset.DateRange()
    def Last_Two_Months(self, metadata):
        return {
            'Start': '2019-11-01',
            'End': '2020-01-01'
        }
    
    @Asset.DateRange()
    def Last_Week(self, metadata):
        return {
            'Start': '2020-01-01',
            'End': '2020-01-08'
        }
        
    @Asset.Document()
    def Topic_Document(self, metadata, topic):
        document = topic.document('Cooling Tower ' + self.definition['Name'])
        document.render_template(filename='Cooling_Tower_Health.html', asset=self)
        
class Region(Asset):    
    @Asset.Component()
    def Cooling_Towers(self, metadata):
        return self.build_components(Cooling_Tower, metadata, column_name='Cooling Tower')

In [4]:
search_df = spy.search({
    'Path': 'Example',
    'Type': 'Signal',
    'Name': 'Temperature'})

search_df

,Name,Path,Type,Time,Count,Result
0,Temperature,Example,Signal,00:00:00.03,10,Success


,ID,Path,Asset,Name,Description,Type,Value Unit Of Measure,Datasource Name,Archived
0,9B3747A1-8014-4DB6-8D7A-349F62ECEE13,Example >> Cooling Tower 1,Area H,Temperature,NaN,StoredSignal,°F,Example Data,False
1,84B78292-7CF9-44BD-8313-3CB793DCC975,Example >> Cooling Tower 1,Area J,Temperature,NaN,StoredSignal,°F,Example Data,False
2,A915291B-DF7F-4B04-BAB6-ED6FD8E40AC0,Example >> Cooling Tower 1,Area I,Temperature,NaN,StoredSignal,°F,Example Data,False
3,8EA020E7-4DC4-4DF8-A63A-696009F24C43,Example >> Cooling Tower 1,Area A,Temperature,NaN,StoredSignal,°F,Example Data,False
4,D0A547F5-FDF3-433F-A214-C1F0B88AD960,Example >> Cooling Tower 1,Area G,Temperature,NaN,StoredSignal,°F,Example Data,False
5,CF7BF2E5-3E51-4F85-85BE-336FFAB2AFEA,Example >> Cooling Tower 1,Area B,Temperature,NaN,StoredSignal,°F,Example Data,False
6,1102EF7F-C1EC-43C4-AE56-B2F859D978F8,Example >> Cooling Tower 2,Area E,Temperature,NaN,StoredSignal,°F,Example Data,False
7,B2C0AD16-1895-4BD5-9071-75C1F7B95A84,Example >> Cooling Tower 2,Area D,Temperature,NaN,StoredSignal,°F,Example Data,False
8,22D698EC-26E2-494B-A7E3-4F86A0B03A6E,Example >> Cooling Tower 1,Area K,Temperature,NaN,StoredSignal,°F,Example Data,False
9,F4C906E3-7BED-42E5-ABBD-6E20AD41C252,Example >> Cooling Tower 1,Area C,Temperature,NaN,StoredSignal,°F,Example Data,False


In [5]:
metadata_df = search_df.copy()
metadata_df['Cooling Tower'] = metadata_df['Path'].str.extract(r'Cooling Tower (\d)')
metadata_df['Area'] = metadata_df['Asset'].str.extract(r'Area (\w)')
metadata_df['Build Path'] = 'Cooling Tower Health'
metadata_df['Build Asset'] = 'North America'
metadata_df

,ID,Path,Asset,Name,Description,Type,Value Unit Of Measure,Datasource Name,Archived,Cooling Tower,Area,Build Path,Build Asset
0,9B3747A1-8014-4DB6-8D7A-349F62ECEE13,Example >> Cooling Tower 1,Area H,Temperature,NaN,StoredSignal,°F,Example Data,False,1,H,Cooling Tower Health,North America
1,84B78292-7CF9-44BD-8313-3CB793DCC975,Example >> Cooling Tower 1,Area J,Temperature,NaN,StoredSignal,°F,Example Data,False,1,J,Cooling Tower Health,North America
2,A915291B-DF7F-4B04-BAB6-ED6FD8E40AC0,Example >> Cooling Tower 1,Area I,Temperature,NaN,StoredSignal,°F,Example Data,False,1,I,Cooling Tower Health,North America
3,8EA020E7-4DC4-4DF8-A63A-696009F24C43,Example >> Cooling Tower 1,Area A,Temperature,NaN,StoredSignal,°F,Example Data,False,1,A,Cooling Tower Health,North America
4,D0A547F5-FDF3-433F-A214-C1F0B88AD960,Example >> Cooling Tower 1,Area G,Temperature,NaN,StoredSignal,°F,Example Data,False,1,G,Cooling Tower Health,North America
5,CF7BF2E5-3E51-4F85-85BE-336FFAB2AFEA,Example >> Cooling Tower 1,Area B,Temperature,NaN,StoredSignal,°F,Example Data,False,1,B,Cooling Tower Health,North America
6,1102EF7F-C1EC-43C4-AE56-B2F859D978F8,Example >> Cooling Tower 2,Area E,Temperature,NaN,StoredSignal,°F,Example Data,False,2,E,Cooling Tower Health,North America
7,B2C0AD16-1895-4BD5-9071-75C1F7B95A84,Example >> Cooling Tower 2,Area D,Temperature,NaN,StoredSignal,°F,Example Data,False,2,D,Cooling Tower Health,North America
8,22D698EC-26E2-494B-A7E3-4F86A0B03A6E,Example >> Cooling Tower 1,Area K,Temperature,NaN,StoredSignal,°F,Example Data,False,1,K,Cooling Tower Health,North America
9,F4C906E3-7BED-42E5-ABBD-6E20AD41C252,Example >> Cooling Tower 1,Area C,Temperature,NaN,StoredSignal,°F,Example Data,False,1,C,Cooling Tower Health,North America


In [6]:
build_df = spy.assets.build(Region, metadata_df)
build_df

,Asset,Asset Object,Build Result,Formula,Formula Parameters,Measured Item,Measured Item Maximum Duration,Name,Object,Path,Statistic,Template,Thresholds,Type,Workbook Type
0,H,Cooling Tower Health >> North America >> 1 >> H,Success,$a.validValues().valueSearch(isGreaterThan(120)),{'$a': ['ID']},NaN,NaN,Incidents,NaN,Cooling Tower Health >> North America >> 1,NaN,Area,NaN,Condition,NaN
1,H,Cooling Tower Health >> North America >> 1 >> H,Success,NaN,NaN,"{'Type': 'Condition', 'Formula': '$a.validValu...",40d,Incident Count,NaN,Cooling Tower Health >> North America >> 1,count,Area,"{'HiHiHi': 1, 'HiHi': 0}",Metric,NaN
2,H,Cooling Tower Health >> North America >> 1 >> H,Success,NaN,NaN,NaN,NaN,H,NaN,Cooling Tower Health >> North America >> 1,NaN,Area,NaN,Asset,NaN
3,J,Cooling Tower Health >> North America >> 1 >> J,Success,$a.validValues().valueSearch(isGreaterThan(120)),{'$a': ['ID']},NaN,NaN,Incidents,NaN,Cooling Tower Health >> North America >> 1,NaN,Area,NaN,Condition,NaN
4,J,Cooling Tower Health >> North America >> 1 >> J,Success,NaN,NaN,"{'Type': 'Condition', 'Formula': '$a.validValu...",40d,Incident Count,NaN,Cooling Tower Health >> North America >> 1,count,Area,"{'HiHiHi': 1, 'HiHi': 0}",Metric,NaN
5,J,Cooling Tower Health >> North America >> 1 >> J,Success,NaN,NaN,NaN,NaN,J,NaN,Cooling Tower Health >> North America >> 1,NaN,Area,NaN,Asset,NaN
6,I,Cooling Tower Health >> North America >> 1 >> I,Success,$a.validValues().valueSearch(isGreaterThan(120)),{'$a': ['ID']},NaN,NaN,Incidents,NaN,Cooling Tower Health >> North America >> 1,NaN,Area,NaN,Condition,NaN
7,I,Cooling Tower Health >> North America >> 1 >> I,Success,NaN,NaN,"{'Type': 'Condition', 'Formula': '$a.validValu...",40d,Incident Count,NaN,Cooling Tower Health >> North America >> 1,count,Area,"{'HiHiHi': 1, 'HiHi': 0}",Metric,NaN
8,I,Cooling Tower Health >> North America >> 1 >> I,Success,NaN,NaN,NaN,NaN,I,NaN,Cooling Tower Health >> North America >> 1,NaN,Area,NaN,Asset,NaN
9,A,Cooling Tower Health >> North America >> 1 >> A,Success,$a.validValues().valueSearch(isGreaterThan(120)),{'$a': ['ID']},NaN,NaN,Incidents,NaN,Cooling Tower Health >> North America >> 1,NaN,Area,NaN,Condition,NaN


In [7]:
spy.push(metadata=build_df)

,Asset,Condition,Overall,Relationship,Scalar,Signal,Threshold Metric
Items pushed,14,10,43,33,0,0,10


,Asset,Asset Object,Build Result,Formula,Formula Parameters,Measured Item,Measured Item Maximum Duration,Name,Object,Path,Statistic,Template,Thresholds,Type,Workbook Type,Datasource Class,Datasource ID,Data ID,ID,Push Result
0,H,Cooling Tower Health >> North America >> 1 >> H,Success,$a.validValues().valueSearch(isGreaterThan(120)),[a=9B3747A1-8014-4DB6-8D7A-349F62ECEE13],NaN,NaN,Incidents,NaN,Cooling Tower Health >> North America >> 1,NaN,Area,NaN,CalculatedCondition,NaN,Seeq Data Lab,Seeq Data Lab,[4ED69886-4ACE-4D2C-96AB-66942208CB35] {Condit...,05D6EB5E-F3DC-4957-B40D-408EEC3DA9E2,Success
1,H,Cooling Tower Health >> North America >> 1 >> H,Success,NaN,NaN,"{'Type': 'Condition', 'Formula': '$a.validValu...",40d,Incident Count,NaN,Cooling Tower Health >> North America >> 1,count,Area,"{'HiHiHi': 1, 'HiHi': 0}",ThresholdMetric,NaN,Seeq Data Lab,Seeq Data Lab,[4ED69886-4ACE-4D2C-96AB-66942208CB35] {Metric...,FC61FF5B-F5DC-43AF-9805-0520EA08574A,Success
2,H,Cooling Tower Health >> North America >> 1 >> H,Success,NaN,NaN,NaN,NaN,H,NaN,Cooling Tower Health >> North America >> 1,NaN,Area,NaN,Asset,NaN,Seeq Data Lab,Seeq Data Lab,[4ED69886-4ACE-4D2C-96AB-66942208CB35] {Asset}...,351B66E7-DE55-45B5-859D-F25592EA59F9,Success
3,J,Cooling Tower Health >> North America >> 1 >> J,Success,$a.validValues().valueSearch(isGreaterThan(120)),[a=84B78292-7CF9-44BD-8313-3CB793DCC975],NaN,NaN,Incidents,NaN,Cooling Tower Health >> North America >> 1,NaN,Area,NaN,CalculatedCondition,NaN,Seeq Data Lab,Seeq Data Lab,[4ED69886-4ACE-4D2C-96AB-66942208CB35] {Condit...,2AD7782F-EE58-4BBA-BFD4-00B93360536B,Success
4,J,Cooling Tower Health >> North America >> 1 >> J,Success,NaN,NaN,"{'Type': 'Condition', 'Formula': '$a.validValu...",40d,Incident Count,NaN,Cooling Tower Health >> North America >> 1,count,Area,"{'HiHiHi': 1, 'HiHi': 0}",ThresholdMetric,NaN,Seeq Data Lab,Seeq Data Lab,[4ED69886-4ACE-4D2C-96AB-66942208CB35] {Metric...,EEC68216-4B15-4D25-92F9-A02FEA373249,Success
5,J,Cooling Tower Health >> North America >> 1 >> J,Success,NaN,NaN,NaN,NaN,J,NaN,Cooling Tower Health >> North America >> 1,NaN,Area,NaN,Asset,NaN,Seeq Data Lab,Seeq Data Lab,[4ED69886-4ACE-4D2C-96AB-66942208CB35] {Asset}...,13A662F5-6114-4FB5-8DC5-6CF74966AB1F,Success
6,I,Cooling Tower Health >> North America >> 1 >> I,Success,$a.validValues().valueSearch(isGreaterThan(120)),[a=A915291B-DF7F-4B04-BAB6-ED6FD8E40AC0],NaN,NaN,Incidents,NaN,Cooling Tower Health >> North America >> 1,NaN,Area,NaN,CalculatedCondition,NaN,Seeq Data Lab,Seeq Data Lab,[4ED69886-4ACE-4D2C-96AB-66942208CB35] {Condit...,41D1CFF0-C192-42DF-B4FD-9A519A818E3F,Success
7,I,Cooling Tower Health >> North America >> 1 >> I,Success,NaN,NaN,"{'Type': 'Condition', 'Formula': '$a.validValu...",40d,Incident Count,NaN,Cooling Tower Health >> North America >> 1,count,Area,"{'HiHiHi': 1, 'HiHi': 0}",ThresholdMetric,NaN,Seeq Data Lab,Seeq Data Lab,[4ED69886-4ACE-4D2C-96AB-66942208CB35] {Metric...,D2E3C61C-4553-4CDB-A11B-7AB73032F2D1,Success
8,I,Cooling Tower Health >> North America >> 1 >> I,Success,NaN,NaN,NaN,NaN,I,NaN,Cooling Tower Health >> North America >> 1,NaN,Area,NaN,Asset,NaN,Seeq Data Lab,Seeq Data Lab,[4ED69886-4ACE-4D2C-96AB-66942208CB35] {Asset}...,53864867-531E-416E-8CFA-CD88D91BEB44,Success
9,A,Cooling Tower Health >> North America >> 1 >> A,Success,$a.validValues().valueSearch(isGreaterThan(120)),[a=8EA020E7-4DC4-4DF8-A63A-696009F24C43],NaN,NaN,Incidents,NaN,Cooling Tower Health >> North America >> 1,NaN,Area,NaN,CalculatedCondition,NaN,Seeq Data Lab,Seeq Data Lab,[4ED69886-4ACE-4D2C-96AB-66942208CB35] {Condit...,4E8E2D4A-5007-4F46-93C9-8ED3666845A6,Success
